In [22]:
# if you haven't install the dependencies
# please note that you might need to install pip first (or use brew instead)

!pip3 install selenium
!pip3 install bs4
!pip3 install pandas

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
     |████████████████████████████████| 4.6 MB 510 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [13]:
# install pandas, beautifulsoup and selenium before if necessary
# imports
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import pandas as pd

In [18]:
# im using chromedriver v96, if not supported download new chromedriver aligned with your chrome version
# create the chrome session 
# please comments one of the code below based on your OS

# FOR MAC
# run this first in terminal for the first time
# xattr -d com.apple.quarantine chromedrivermac
import os
os.chmod('./chromedriver.exe', 0o755)
driver = webdriver.Chrome(executable_path="./chromedrivermac")

# FOR WINDOWS
# driver = webdriver.Chrome(executable_path="chromedriver.exe")

/var/folders/v9/q_svsx3d1b9966jt8fzvt_njd6xb7g/T/ipykernel_33299/1621861197.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedrivermac")


In [24]:
# wait time between page loaded and scraped (in seconds) to handle slow internet
pageLoadWait = 5

# open the page and save it variable content then make it readable using beautifulsoup
def getListProductPage(pageNum):
    driver.get("https://www.tokopedia.com/p/handphone-tablet/handphone?page=" + str(pageNum))
    driver.find_element_by_tag_name('html').send_keys(Keys.END)

    time.sleep(pageLoadWait)
    content = driver.page_source
    return BeautifulSoup(content)

In [20]:
# tokopedia have promotion program, some links start with ta.tokopedia
# it doesn't allow direct call for the promotion link so we need to extract the standard link
def getLink(card):
    link = card["href"]
    if (link.startswith("https://ta.tokopedia.com")):
        link = link[link.index("www.tokopedia.com"):link.index("%3Fsrc")]
        link = "https://" + link.replace('%2F', '/')
    return link

# we take the shop name from the list page because it's done async at product page
# sometimes some card have image of shop before shop name so need to handle that as well
def getShopName(card):
    wrapper = list(list(card.find("div", {"data-testid": "divProductWrapper"}))[1])[2]
    if len(list(list(wrapper)[0])) > 1:
        return list(list(wrapper)[0])[1].getText()
    else:
        return list(list(wrapper)[1])[1].getText()

In [25]:
# tokopedia have tags for their automated testing called "data-testid", we will use those tags for reference
numberOfProduct = 100

data = {}
i = 1
while (len(data.keys())<numberOfProduct):
    for card in getListProductPage(i).findAll("a", {"data-testid": "lnkProductContainer"}):
        data[getLink(card)] = getShopName(card)
        if len(data.keys())==numberOfProduct:
            break
    i+=1
data

/var/folders/v9/q_svsx3d1b9966jt8fzvt_njd6xb7g/T/ipykernel_33299/653648952.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_tag_name('html').send_keys(Keys.END)


{'https://www.tokopedia.com/sodo-shop/hp-samsung-lipat-e1272-termurah-dual-sim-2-garansi-distributor-hitam-a056': 'sodo shop',
 'https://www.tokopedia.com/fantasticmart/nokia-105-2019-grs-tam-biru': 'Fantasticmart',
 'https://www.tokopedia.com/sunmobile22/iphone-8-second-red-64-gb': 'sun mobile',
 'https://www.tokopedia.com/sodo-shop/new-nokia-1280-ready-6-warna-hitam-ac16': 'sodo shop',
 'https://www.tokopedia.com/sunmobile22/iphone-xr-second-yellow-64-gb': 'sun mobile',
 'https://www.tokopedia.com/shopyzid/samsung-galaxy-a32-5g-nfc-8gb-128gb-8-128-garansi-resmi-awesome-black?whid=0': 'Shopyz ID',
 'https://www.tokopedia.com/xiaomi/xiaomi-mi-poco-m3-pro-5g-4-64gb-nfc-triple-kamera-layar-smartphone-hp-cool-blue?whid=0': 'Xiaomi Official Store',
 'https://www.tokopedia.com/xiaomi/xiaomi-mi-poco-x3-pro-6-128gb-nfc-snapdragon-860-48mp-5160mah-metal-bronze?whid=0': 'Xiaomi Official Store',
 'https://www.tokopedia.com/gudang-hp/samsung-galaxy-a12-6-128-gb-garansi-resmi-sein-putih?whid=0': '

In [26]:
# functions to get details from detail page
def getName(body):
    return body.find("h1", {"data-testid": "lblPDPDetailProductName"}).getText()

def getDesc(body):
    result = ""
    for string in body.find("div", {"data-testid": "lblPDPDescriptionProduk"}).strings:
        result += string + "\n"
    return result

def getImg(body):
    imgContainer = body.find("div", {"data-testid": "PDPImageMain"})
    return list(list(list(imgContainer)[0])[1])[0]["src"]

# it's possible that there's no rating so will return null
def getRating(body):
    ratingText = body.find("span", {"data-testid": "lblPDPDetailProductRatingNumber"})
    if (ratingText is not None):
        return float(ratingText.getText())

def getPrice(body):
    textPrice = body.find("div", {"data-testid": "lblPDPDetailProductPrice"}).getText()
    return int(re.sub('[^0-9]','', textPrice))

In [ ]:
# because we need description, we have to open the pages of each product to get it

columns = ["name","desc","imgUrl","price","rating","storeName"]
products = []
for link in data:
    driver.get(link)
    time.sleep(pageLoadWait)
    content = driver.page_source
    soup = BeautifulSoup(content)
    products.append([getName(soup), getDesc(soup), getImg(soup), getPrice(soup), getRating(soup), data[link]])
products

In [10]:
df = pd.DataFrame(products, columns=columns)
df

,name,desc,imgUrl,price,rating,storeName
0,hp samsung e1272 termurah original jadul samsu...,Samsung dual card dual standby ponsel flip 127...,https://images.tokopedia.net/img/cache/500-squ...,153500,4.6,Little Master.shop
1,nokia jadul 105 dual sim 2017 - Hitam,Detail produk dari NOKIA 105 DUAL SIM (TA-1174...,https://images.tokopedia.net/img/cache/500-squ...,119000,4.8,Kkeep
2,OPPO F5 4/64GB TERPERCAYA GARANSI 1TAHUN - 4/6...,**BUDAYAKAN MEMBACA DESKRIPSI SEBELUM MEMBELI*...,https://images.tokopedia.net/img/cache/500-squ...,999000,4.9,Trust Phonestore
3,"IPHONE XR SECOND - Red, 64 gb",SELAMAT DATANG DI Sun MOBILE\nHarap Menanyaka...,https://images.tokopedia.net/img/cache/500-squ...,4750000,4.9,sun mobile
4,iPhone 13 Pro Max 128GB 256GB 512GB 1TB Garans...,MARKAS APPLE\nMarkas apple memberikan solusi u...,https://images.tokopedia.net/img/cache/500-squ...,21499000,4.8,MARKAS APPLE
...,...,...,...,...,...,...
95,SAMSUNG Galaxy A32 [8/128GB] - Garansi Resmi S...,SAMSUNG Galaxy A32 [8/128GB] - Garansi Resmi S...,https://images.tokopedia.net/img/cache/500-squ...,3625000,5.0,Rejodadi Surabaya
96,INFINIX SMART 5 RAM 2 32 GARANSI RESMI - NON GIFT,"GARANSI RESMI INFINIX INDONESIA,IMEI TERDAFTAR...",https://images.tokopedia.net/img/cache/500-squ...,1299000,5.0,Bintang Terang Cell
97,REDMI NOTE 10S RAM 8 128 NFC RAM 6 64 NFC GARA...,"GARANSI RESMI XIAOMI INDONESIA,IMEI TERDAFTAR\...",https://images.tokopedia.net/img/cache/500-squ...,2675000,4.9,Bintang Terang Cell
98,Xiaomi Poco X3 Pro Ram 8 Rom 256 GB Garansi Re...,NETWORK Technology\nGSM / HSPA / LTE\nLAUNCH A...,https://images.tokopedia.net/img/cache/500-squ...,3833000,4.9,Best Phone Cell


In [12]:
df.to_csv(r'scrapResult.csv', sep = ';')